In [ ]:
import pandas as pd
import oemof.solph as solph
from oemof.tools import economics
import numpy as np

# Load your data
data = pd.read_csv('your_data.csv')
mean_demand = data['demand'].mean()
std_demand = data['demand'].std()
mean_solar = data['solar_generation'].mean()
std_solar = data['solar_generation'].std()

battery_costs_per_kWh = 100  # Placeholder, adjust accordingly
n_scenarios = 1000  # Number of scenarios to generate

best_cost = float('inf')
best_capacity = 0

# Loop over a range of potential storage capacities
for capacity in range(100, 10000, 100):
    total_cost_scenario = 0

    for _ in range(n_scenarios):
        # Generate a scenario
        scenario_demand = np.random.normal(mean_demand, std_demand, len(data))
        scenario_solar = np.random.normal(mean_solar, std_solar, len(data))

        # Create the energy system and optimization model for the scenario
        es = solph.EnergySystem(timeindex=data.index)
        bel = solph.Bus(label='electricity')
        es.add(bel)

        es.add(solph.Source(label='grid', outputs={bel: solph.Flow(variable_costs=data['real_time_prices']}))
        es.add(solph.Source(label='solar', outputs={bel: solph.Flow(actual_value=scenario_solar, fixed=True)}))
        es.add(solph.Sink(label='demand', inputs={bel: solph.Flow(nominal_value=scenario_demand.max(), actual_value=scenario_demand, fixed=True)}))
        es.add(solph.Storage(label='battery', nominal_storage_capacity=capacity, inputs={bel: solph.Flow()},
                            outputs={bel: solph.Flow()}))

        # Solve the optimization model for the scenario
        om = solph.Model(es)
        om.solve(solver='cbc')

        # Calculate the net cost for this scenario and battery capacity
        grid_flow = solph.views.node(om.results, 'grid')['sequences']
        total_cost_scenario += sum(grid_flow['flow']) * data['real_time_prices'].mean()

    # Average the cost over all scenarios
    average_cost_scenario = total_cost_scenario / n_scenarios
    battery_investment = economics.annuity(capex=battery_costs_per_kWh*capacity, n=20, wacc=0.05)
    average_cost_scenario += battery_investment

    if average_cost_scenario < best_cost:
        best_cost = average_cost_scenario
        best_capacity = capacity

print(f"The best battery capacity to install, considering the stochastic properties, is: {best_capacity} kWh")
